In [ ]:
import uuid

import numpy as np
import plotly.graph_objects as go

from ipywidgets import interact

In [ ]:
def R(x, z_a=1, z_b=0, L=1, return_dydx=False):
    y = z_b + (z_a - z_b) * ((x - L) / L) ** 2

    if return_dydx:
        dydx = (2 / L) * (z_a - z_b) * ((x - L) / L)
        return y, dydx
    else:
        return y


def B(x, z_a=1, z_b=0, L=1, return_dydx=False):
    y = z_a - (z_a - z_b) * (x / L)

    if return_dydx:
        dydx = (z_b - z_a) * np.ones_like(x) / L
        return y, dydx
    else:
        return y

In [ ]:
x = np.linspace(0, 1)

trace_R = go.Scatter(
    x=x,
    y=R(x),
    fill="tozeroy",
    name="R",
    mode="lines",
    line_color="red",
)

trace_B = go.Scatter(
    x=x,
    y=B(x),
    fill="tonexty",
    name="B",
    mode="lines",
    line_color="blue",
)

trace_Z = go.Scatter(
    x=[0, 1],
    y=[1, 0],
    mode="markers+text",
    name="Départ et arrivée",
    text=[r"$z_A$", r"$z_B$"],
    textposition=["top center", "middle right"],
    line_color="green",
)

fig_1 = go.FigureWidget(data=[trace_R, trace_B, trace_Z])


@interact(L=(1, 10, 1), z_a=(0, 1, 0.1), z_b=(0, 1, 0.1))
def update(L=1, z_a=1, z_b=0):
    with fig_1.batch_update():
        x = np.linspace(0, L)
        fig_1.data[0].x = x
        fig_1.data[0].y = R(x, L=L, z_a=z_a, z_b=z_b)
        fig_1.data[1].x = x
        fig_1.data[1].y = B(x, L=L, z_a=z_a, z_b=z_b)
        fig_1.data[2].x = [0, L]
        fig_1.data[2].y = [z_a, z_b]
        fig_1.data[2].uid = str(uuid.uuid4())  # This forces the text to be updated


fig_1

In [ ]:
x = np.linspace(0, 1)

trace_R = go.Scatter(
    x=x,
    y=R(x),
    name="R",
    mode="lines",
    line_color="red",
)

trace_B = go.Scatter(
    x=x,
    y=B(x),
    name="B",
    mode="lines",
    line_color="blue",
)

trace_R_approx = go.Bar(
    x=x,
    y=R(x),
    offset=0.5 * (x[1] - x[0]),
    width=x[1] - x[0],
    marker_color="red",
    name="Approximation de R",
    opacity=0.5,
)

trace_B_approx = go.Bar(
    x=x,
    y=B(x),
    offset=0.5 * (x[1] - x[0]),
    width=x[1] - x[0],
    marker_color="blue",
    name="Approximation de B",
    opacity=0.5,
)

trace_Z = go.Scatter(
    x=[0, 1],
    y=[1, 0],
    mode="markers+text",
    name="Départ et arrivée",
    text=[r"$z_A$", r"$z_B$"],
    textposition=["top center", "middle right"],
    line_color="green",
)

fig_2 = go.FigureWidget(
    data=[trace_B_approx, trace_R_approx, trace_R, trace_B, trace_Z]
)


def aire(x: np.ndarray, y: np.ndarray) -> float:
    """
    Calculer l'aire sous la courbe, en utilisant l'intégrale
    par somme des rectangles (ou [BONUS] trapèzes).

    x: vecteur des abscisses, de 0 à L, de longueur n
    y: vecteur de ordonnées, de longeur n
    """
    dx = x[1] - x[0]  # Base d'un rectangle

    # return dx * np.sum(y[:-1])
    return np.trapz(y, dx=dx)
    return 0  # À changer !


def travail(x: np.ndarray, y: np.ndarray, dydx: np.ndarray, F: np.ndarray) -> float:
    """
    Calcul le travail effectué par F le long d'une courbe.

    x: vecteur des abscisses, de 0 à L, de longueur n
    y: vecteur de ordonnées, de longeur n
    dydx: vecteur des dérivées % à x, de longueur n
    F: vecteur force, de longeur 2, i.e., F = [F_x, F_y]
    """
    dx = x[1] - x[0]  # Base d'un rectangle
    x = x[:-1]
    n = len(x)
    df = np.zeros((2, n))
    df[0, :] = np.ones(n)
    df[1, :] = dydx[:-1]

    F_proj = np.dot(F, df)

    # return np.sum(F_proj[:-1] * dx)
    return np.trapz(F_proj, dx=dx)


g = np.array([0, -9.81])
m = 1
F = m * g


@interact(L=(1, 10, 1), z_a=(0, 1, 0.1), z_b=(0, 1, 0.1), n=(4, 100, 4))
def update(L=1, z_a=1, z_b=0, n=4):
    with fig_2.batch_update():
        x = np.linspace(0, L)
        fig_2.data[2].x = x
        fig_2.data[2].y = R(x, L=L, z_a=z_a, z_b=z_b)
        fig_2.data[3].x = x
        fig_2.data[3].y = B(x, L=L, z_a=z_a, z_b=z_b)
        x, dx = np.linspace(0, L, n, retstep=True)
        y_R, dydx_R = R(x, L=L, z_a=z_a, z_b=z_b, return_dydx=True)
        y_B, dydx_B = B(x, L=L, z_a=z_a, z_b=z_b, return_dydx=True)
        fig_2.data[1].x = x[:-1]
        fig_2.data[1].y = y_R[:-1]
        fig_2.data[1].width = dx
        fig_2.data[0].x = x[:-1]
        fig_2.data[0].y = y_B[:-1]
        fig_2.data[0].width = dx
        fig_2.data[-1].x = [0, L]
        fig_2.data[-1].y = [z_a, z_b]
        fig_2.data[-1].uid = str(uuid.uuid4())  # This forces the text to be updated
        aire_R = aire(x, y_R)
        aire_B = aire(x, y_B)
        travail_R = travail(x, y_R, dydx_R, F)
        travail_B = travail(x, y_B, dydx_B, F)
        fig_2.layout.title.text = f"Aire R, B : {aire_R:.4}, {aire_B:.4} / Travail R, B : {travail_R:.4}, {travail_B:.4}"


fig_2